In [1]:
import shap
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

def get_iris_data():
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]
    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data
    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

# Example that generates nans

In [2]:
train_X, test_X,_,_ = get_iris_data()
# train_X = train_X[:,1:5]
# test_X = test_X[:,1:5]
inds = np.random.choice(train_X.shape[0], 3, replace=False)
data = train_X[inds,:]
test_in = test_X[10:11,:]
x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
X = tf.placeholder("float", shape=[None, x_size])

In [3]:
print(train_X)

[[1.  5.7 2.9 4.2 1.3]
 [1.  7.6 3.  6.6 2.1]
 [1.  5.6 3.  4.5 1.5]
 [1.  5.1 3.5 1.4 0.2]
 [1.  7.7 2.8 6.7 2. ]
 [1.  5.8 2.7 4.1 1. ]
 [1.  5.2 3.4 1.4 0.2]
 [1.  5.  3.5 1.3 0.3]
 [1.  5.1 3.8 1.9 0.4]
 [1.  5.  2.  3.5 1. ]
 [1.  6.3 2.7 4.9 1.8]
 [1.  4.8 3.4 1.9 0.2]
 [1.  5.  3.  1.6 0.2]
 [1.  5.1 3.3 1.7 0.5]
 [1.  5.6 2.7 4.2 1.3]
 [1.  5.1 3.4 1.5 0.2]
 [1.  5.7 3.  4.2 1.2]
 [1.  7.7 3.8 6.7 2.2]
 [1.  4.6 3.2 1.4 0.2]
 [1.  6.2 2.9 4.3 1.3]
 [1.  5.7 2.5 5.  2. ]
 [1.  5.5 4.2 1.4 0.2]
 [1.  6.  3.  4.8 1.8]
 [1.  5.8 2.7 5.1 1.9]
 [1.  6.  2.2 4.  1. ]
 [1.  5.4 3.  4.5 1.5]
 [1.  6.2 3.4 5.4 2.3]
 [1.  5.5 2.3 4.  1.3]
 [1.  5.4 3.9 1.7 0.4]
 [1.  5.  2.3 3.3 1. ]
 [1.  6.4 2.7 5.3 1.9]
 [1.  5.  3.3 1.4 0.2]
 [1.  5.  3.2 1.2 0.2]
 [1.  5.5 2.4 3.8 1.1]
 [1.  6.7 3.  5.  1.7]
 [1.  4.9 3.1 1.5 0.1]
 [1.  5.8 2.8 5.1 2.4]
 [1.  5.  3.4 1.5 0.2]
 [1.  5.  3.5 1.6 0.6]
 [1.  5.9 3.2 4.8 1.8]
 [1.  5.1 2.5 3.  1.1]
 [1.  6.9 3.2 5.7 2.3]
 [1.  6.  2.7 5.1 1.6]
 [1.  6.1 2

### Multiplication

In [8]:
# yhat = tf.multiply(X[:,0:1],X)
# yhat = tf.multiply(X,X[:,0:1])
yhat = tf.multiply(X,X)
model = (X,yhat)
e = shap.DeepExplainer(model, data)
shap_values = e.shap_values(test_in)
sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
sess = tf.Session()
diff = sess.run(model[1], feed_dict={model[0]: test_in})[0,:] - \
       sess.run(model[1], feed_dict={model[0]: data}).mean(0)
print(sums)
print(diff)
assert np.allclose(sums, diff, atol=1e-06), "Sum of SHAP values does not match difference!"

[ 0.         11.59333391  1.03666701 14.87333215  2.50333347]
[ 0.        11.593332   1.0366669 14.87333    2.5033336]


### Division

In [9]:
# yhat = tf.div(X,X[:,0:1]) # These examples don't work
# yhat = tf.div(X[:,0:1],X) # These examples don't work
# yhat = tf.div(X[:,0:2],X[:,2:4])
yhat = tf.div(X,X)
model = (X,yhat)
e = shap.DeepExplainer(model, data)
shap_values = e.shap_values(test_in)
sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
sess = tf.Session()
diff = sess.run(model[1], feed_dict={model[0]: test_in})[0,:] - \
       sess.run(model[1], feed_dict={model[0]: data}).mean(0)
print(sums)
print(diff)
assert np.allclose(sums, diff, atol=1e-06), "Sum of SHAP values does not match difference!"

[0.00000000e+00 2.98023224e-09 9.93410746e-09 0.00000000e+00
 0.00000000e+00]
[0. 0. 0. 0. 0.]
